In [3]:
# =============================================
# 1. Librerías
# =============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer

# =============================================
# 2. Cargar datos (asegúrate que estos son los únicos cargados)
# =============================================
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print("Shape train:", train.shape)
print("Shape test :", test.shape)

# =============================================
# 3. Nos quedamos SOLO con la columna que venimos trabajando
# =============================================
train = train[['ID','FAMI_ESTRATOVIVIENDA','RENDIMIENTO_GLOBAL']]
test = test[['ID','FAMI_ESTRATOVIVIENDA']]

# =============================================
# 4. Imputación de faltantes con 'Estrato 2'
# =============================================
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].fillna("Estrato 2")
test['FAMI_ESTRATOVIVIENDA'] = test['FAMI_ESTRATOVIVIENDA'].fillna("Estrato 2")

# =============================================
# 5. One-hot encoding de FAMI_ESTRATOVIVIENDA
# =============================================
def one_hot(df, col):
    dummies = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df.drop(columns=[col]), dummies], axis=1)
    return df

train = one_hot(train, 'FAMI_ESTRATOVIVIENDA')
test = one_hot(test, 'FAMI_ESTRATOVIVIENDA')

# Alineamos test al train
test = test.reindex(columns=train.drop(columns=['RENDIMIENTO_GLOBAL']).columns, fill_value=0)

# =============================================
# 6. Conversión de target a valores numéricos
# =============================================
mapa = {'bajo':0, 'medio-bajo':1, 'medio-alto':2, 'alto':3}
y = train['RENDIMIENTO_GLOBAL'].map(mapa)
X = train.drop(columns=['RENDIMIENTO_GLOBAL'])

# =============================================
# 7. Modelo y validación
# =============================================
estimator = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
cv = ShuffleSplit(n_splits=30, test_size=0.3, random_state=42)

scores = cross_val_score(estimator, X, y, cv=cv, scoring=make_scorer(mean_absolute_error))
print("MAE promedio: %.3f (±%.3f)" % (scores.mean(), scores.std()))

# =============================================
# 8. Entrenamiento final y predicción
# =============================================
estimator.fit(X, y)
y_pred = estimator.predict(test)

# =============================================
# 9. Redondeo y desmapear a labels
# =============================================
y_pred_labels = np.round(y_pred).astype(int)
inv_mapa = {v:k for k,v in mapa.items()}
y_pred_labels = pd.Series(y_pred_labels).map(inv_mapa)

# =============================================
# 10. Submission
# =============================================
submission = pd.DataFrame({
    'ID': test['ID'],
    'RENDIMIENTO_GLOBAL': y_pred_labels
})

submission.to_csv("submission.csv", index=False)
print(submission.head())


Shape train: (692500, 12)
Shape test : (296786, 12)
MAE promedio: 0.942 (±0.001)
       ID RENDIMIENTO_GLOBAL
0  550236         medio-alto
1   98545         medio-bajo
2  499179         medio-alto
3  782980         medio-bajo
4  785185         medio-bajo
